<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/DodgersSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 40.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b48e091dc66b99c47ef0b74beacb00468fc3cb15e0325e4c2e5ed6dd1b45d6f1
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
import pyspark as py

In [5]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.data

--2021-04-01 02:31:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961432 (939K) [application/x-httpd-php]
Saving to: ‘Dodgers.data’

Dodgers.data        100%[===================>] 938.90K  1.17MB/s    in 0.8s    

2021-04-01 02:31:36 (1.17 MB/s) - ‘Dodgers.data’ saved [961432/961432]



In [7]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.events

--2021-04-01 02:32:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.events
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4140 (4.0K) [application/x-httpd-php]
Saving to: ‘Dodgers.events’

Dodgers.events      100%[===================>]   4.04K  --.-KB/s    in 0s      

2021-04-01 02:32:04 (83.2 MB/s) - ‘Dodgers.events’ saved [4140/4140]



Attribute Information:

1. Date: MM/DD/YY
2. Time: (H)H:MM (military time)
3. Count: Number of cars measured for the previous five minutes
Rows: Each five minute time slice is represented by one row

For .events file:
1. Date: MM/DD/YY
2. Begin event time: HH:MM:SS (military)
3. End event time: HH:MM:SS (military)
4. Game attendance
5. Away team
6. W/L score

In [11]:
sc=py.SparkContext()

In [9]:
trafficPath="Dodgers.data"
gamesPath="Dodgers.events"

In [12]:
traffic=sc.textFile(trafficPath)

In [13]:
traffic.take(10)

['4/10/2005 0:00,-1',
 '4/10/2005 0:05,-1',
 '4/10/2005 0:10,-1',
 '4/10/2005 0:15,-1',
 '4/10/2005 0:20,-1',
 '4/10/2005 0:25,-1',
 '4/10/2005 0:30,-1',
 '4/10/2005 0:35,-1',
 '4/10/2005 0:40,-1',
 '4/10/2005 0:45,-1']

In [14]:
games=sc.textFile(gamesPath)
games.take(10)

['04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�',
 '04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-1�',
 '04/15/05,19:40:00,21:48:00,51816,San Diego,W 4-0�',
 '04/16/05,19:10:00,21:52:00,54704,San Diego,W 8-3�',
 '04/17/05,13:10:00,15:31:00,53402,San Diego,W 6-0�',
 '04/25/05,19:10:00,21:33:00,36876,Arizona,L 4-2�',
 '04/26/05,19:10:00,22:00:00,44486,Arizona,L 3-2�',
 '04/27/05,19:10:00,22:17:00,54387,Arizona,L 6-3�',
 '04/29/05,19:40:00,22:01:00,40150,Colorado,W 6-3�',
 '04/30/05,19:10:00,21:45:00,54123,Colorado,W 6-2�']

In [16]:
 from datetime import datetime
 import csv
 from io import StringIO 

In [18]:
def parseTraffic(row):
  DATE_FMT= "%m/%d/%Y %H:%M"
  row=row.split(",")
  row[0]=datetime.strptime(row[0],DATE_FMT)
  row[1]=int(row[1])
  return(row)

In [21]:
trafficParsed = traffic.map(parseTraffic)
trafficParsed.take(10)

[[datetime.datetime(2005, 4, 10, 0, 0), -1],
 [datetime.datetime(2005, 4, 10, 0, 5), -1],
 [datetime.datetime(2005, 4, 10, 0, 10), -1],
 [datetime.datetime(2005, 4, 10, 0, 15), -1],
 [datetime.datetime(2005, 4, 10, 0, 20), -1],
 [datetime.datetime(2005, 4, 10, 0, 25), -1],
 [datetime.datetime(2005, 4, 10, 0, 30), -1],
 [datetime.datetime(2005, 4, 10, 0, 35), -1],
 [datetime.datetime(2005, 4, 10, 0, 40), -1],
 [datetime.datetime(2005, 4, 10, 0, 45), -1]]

In [26]:
dailyTrend=trafficParsed.map(lambda x:(x[0].date(),x[1])).reduceByKey(lambda x,y:x+y)
dailyTrend.take(10)

[(datetime.date(2005, 4, 10), -288),
 (datetime.date(2005, 4, 11), 5062),
 (datetime.date(2005, 4, 14), 6423),
 (datetime.date(2005, 4, 15), 6459),
 (datetime.date(2005, 4, 16), 6002),
 (datetime.date(2005, 4, 17), 5322),
 (datetime.date(2005, 4, 18), 5600),
 (datetime.date(2005, 4, 19), 6049),
 (datetime.date(2005, 4, 21), 5977),
 (datetime.date(2005, 4, 22), 6038)]